<a href="https://colab.research.google.com/github/dpnolan/AI/blob/master/Low_Level_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 Introduction

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'1.12.0'

In [0]:
a=3. # a rank 0 tensor; a scalar with shape [],
b=[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
c=[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
d=[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

In [8]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [9]:
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [0]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

In [11]:
!pwd

/content


In [13]:
!ls -altr

total 20
drwxr-xr-x 1 root root 4096 Nov 29 18:21 .config
drwxr-xr-x 2 root root 4096 Nov 29 18:21 sample_data
drwxr-xr-x 1 root root 4096 Dec  1 15:40 ..
drwxr-xr-x 1 root root 4096 Dec  1 16:05 .
-rw-r--r-- 1 root root  287 Dec  1 16:05 events.out.tfevents.1543680356.5e35b18a08df


In [20]:
sess = tf.Session()
print(sess.run(total))

7.0


In [22]:
print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


In [24]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.8610325  0.12572145 0.53890204]
[0.3044281  0.5547668  0.99774504]
(array([1.0062814, 1.5588205, 1.6875824], dtype=float32), array([2.0062814, 2.5588205, 2.6875825], dtype=float32))


In [0]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [27]:
z

<tf.Tensor 'add_5:0' shape=<unknown> dtype=float32>

In [0]:
z1=(sess.run(z, feed_dict={x: 3, y: 4.5}))
z2=(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))


In [33]:
z1

7.5

In [34]:
z2

array([3., 7.], dtype=float32)

In [0]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

In [37]:
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break


[0 1]
[2 3]
[4 5]
[6 7]


In [45]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[-1.6936624 -1.1397572  2.9937596]
[-0.9362539  -0.37387708 -0.9409202 ]
[ 2.1415143   1.152487   -0.13004439]
[-1.2388612 -0.6519705  2.0890203]
[ 0.59097886  0.2892903  -0.5117228 ]
[1.2881304  0.30884984 0.11829697]
[-0.9345663   0.392481   -0.23887293]
[0.13957448 0.49536616 0.47892746]
[ 1.4644263  -0.39770544  1.4645631 ]
[0.05948121 1.1183243  1.5249916 ]


In [0]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)


In [0]:
init = tf.global_variables_initializer()
sess.run(init)

In [56]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))


[[2.1192846]
 [3.2077045]]


In [57]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))


[[1.2565318]
 [0.6305063]]


In [58]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [59]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

In [60]:
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


In [0]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

In [0]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

In [64]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[0.83590305]
 [1.6718061 ]
 [2.507709  ]
 [3.3436122 ]]


In [65]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))


17.099535


In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [0]:
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

In [68]:
print(sess.run(y_pred))

[[-0.71187794]
 [-1.3449537 ]
 [-1.9780293 ]
 [-2.611105  ]]
